In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("allenai/macaw-large")
model = AutoModelForSeq2SeqLM.from_pretrained("allenai/macaw-large")
input_string = "$answer$ ; $mcoptions$  = (A) blue (B) white (C) grey (D) white' ; $question$ = What is the color of a cloudy sky?"
input_ids = tokenizer.encode(input_string, return_tensors="pt")
output = model.generate(input_ids, max_length=200)
print(tokenizer.batch_decode(output, skip_special_tokens=True))
model.cuda()

['$answer$ = grey']


T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=1024, out_features=4096, bias=False)
              (wo): Linear(in_features=4096, out_features=1024, bias=False)
              (

In [2]:
%cd /home/admiraldarius

#!wget https://ai2-public-datasets.s3.amazonaws.com/beliefbank/beliefbank-data-sep2021.zip
#!unzip beliefbank-data-sep2021.zip 


import json
f=open("calibration_facts.json")
f=json.load(f)

calibration_facts=[]
calibration_facts_labels=[]
for i in f.keys():
    for j in f[i]:
        calibration_facts.append(i+" "+j.replace(","," ").replace("IsA","is a").replace("CapableOf","is capable of"))
        calibration_facts_labels.append(f[i][j])
print("exampless of calibration facts:") 
print(calibration_facts[:5],calibration_facts_labels[:5])
print("size of calibration facts:",len(calibration_facts)) 

f=open("silver_facts.json")
f=json.load(f)

silver_facts=[]
silver_facts_labels=[]
for i in f.keys():
    for j in f[i]:
        silver_facts.append(i+" "+j.replace(","," ").replace("IsA","is a").replace("CapableOf","is capable of"))
        silver_facts_labels.append(f[i][j])
        
print("\nexampless of silve facts:")    
print(silver_facts[:5],silver_facts_labels[:5])
print("size of silve facts:",len(silver_facts))

import time
t=time.time()
import torch
input_data=[]
correct,incorrect=0,0
for num,sample in enumerate(calibration_facts):
    input_data.append(tokenizer.encode("$answer$ ; $mcoptions$  = (A) yes (B) no ; $question$ = "\
                                       +sample+"?",return_tensors="pt").cuda())

for num,i in enumerate(input_data):
    output=tokenizer.batch_decode(model.generate(i, max_length=200), skip_special_tokens=True)[0].split("=")[1].strip()
    if output==calibration_facts_labels[num]:
        correct+=1
    else:
        incorrect+=1
        #print(calibration_facts[num])
    if (correct+incorrect)%100==0:
        print(correct/(correct+incorrect))
        print(time.time()-t)
print("final",correct/(correct+incorrect))

/home/admiraldarius
exampless of calibration facts:
['albatross is a bird', 'albatross is a seabird', 'albatross is a animal', 'albatross is a eukaryotic_organism', 'albatross is a pelagic_bird'] ['yes', 'yes', 'yes', 'yes', 'yes']
size of calibration facts: 1072

exampless of silve facts:
['american bison is a mammal', 'american bison is a american bison', 'american bison is a animal', 'american bison is a vertebrate', 'american bison is a warm blooded animal'] ['yes', 'yes', 'yes', 'yes', 'yes']
size of silve facts: 12636
0.68
16.631850004196167
0.69
33.15642333030701
0.7366666666666667
49.63941955566406
0.7625
66.14401626586914
0.696
82.62430620193481
0.6283333333333333
99.12261605262756
0.6042857142857143
115.59577798843384
0.61625
132.0911259651184
0.5866666666666667
148.56211137771606
0.587
165.041029214859
final 0.5867537313432836


In [ ]:
import time
t=time.time()
import torch
input_data=[]
correct,incorrect=0,0
for num,sample in enumerate(silver_facts):
    input_data.append(tokenizer.encode("$answer$ ; $mcoptions$  = (A) yes (B) no ; $question$ = "\
                                       +sample+"?",return_tensors="pt").cuda())

for num,i in enumerate(input_data):
    output=tokenizer.batch_decode(model.generate(i, max_length=200), skip_special_tokens=True)[0].split("=")[1].strip()
    if output==silver_facts_labels[num]:
        correct+=1
    else:
        incorrect+=1
        #print(silver_facts[num])
    if (correct+incorrect)%100==0:
        print(correct/(correct+incorrect))
        print(time.time()-t)
print("final",correct/(correct+incorrect))

0.47
18.201756477355957
0.5
34.831063747406006
0.49666666666666665
51.44942092895508
0.42
68.072993516922
0.45
84.67941188812256
0.4533333333333333
101.31402492523193
0.43142857142857144
117.90821957588196
0.44875
134.53002882003784
0.46111111111111114
151.12114453315735
0.492
167.73890376091003
0.4918181818181818
184.33173990249634
0.5341666666666667
200.9232759475708
0.5292307692307693
217.50550246238708
0.5285714285714286
234.085999250412
0.5393333333333333
250.66305136680603
0.54375
267.22389101982117
0.5494117647058824
283.80786967277527
0.5383333333333333
300.3627619743347
0.5352631578947369
316.9244349002838
0.5405
333.4664297103882
0.5433333333333333
350.0254740715027
0.5281818181818182
366.57157945632935
0.5304347826086957
383.1365406513214
0.5445833333333333
399.70464730262756
0.5556
416.2578067779541
0.5507692307692308
432.7786605358124
0.5503703703703704
449.32168555259705
0.5439285714285714
465.86219024658203
0.5441379310344827
482.4158444404602
0.539
498.981365442276
0.54